In [2]:
import torch
from sklearn.model_selection import train_test_split
from transformers import BertModel
from torch import nn
import pandas as pd
from transformers import BertTokenizer
from nets_bert import BertClassifier
from utils_bert import Dataset_2, marich
device = torch.device("cuda:1")

In [3]:
test = pd.read_csv("./bert_data/bbc/bbc_train.csv")

In [4]:
train, test =train_test_split(test, test_size = 290)

In [5]:
import numpy as np
np.unique(train["Category"])

array(['business', 'entertainment', 'politics', 'sport', 'tech'],
      dtype=object)

In [6]:
labels_dict = {np.unique(train["Category"])[i]:i for i in range(5)}

In [7]:
# train_text = train["Text"]
# train_labels = np.array(train["Category"].map(labels_dict))

# val_text = val["Text"]
# val_labels = np.array(val["Category"].map(labels_dict))

test_text = test["Text"]
test_labels = np.array(test["Category"].map(labels_dict))

In [8]:
test_text = test_text.apply(lambda x: x.replace("\\", " "))

In [9]:
test_data = Dataset_2(test_text, test_labels)

In [10]:
# train_data = torch.load("ag_train.pt")
val_data = torch.load("ag_val.pt")
import pickle
with open('ag_train_unbal_x.pkl', 'rb') as f:
    train_x = pickle.load(f)
with open('ag_train_unbal_y.pkl', 'rb') as f:
    train_y = pickle.load(f)

In [11]:
train_data = Dataset_2(train_x, train_y)

In [12]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, random_split
batch_size = 8

# train_data, validation_data = random_split(train_data, [1000, 200])
# train_sampler = RandomSampler(train_data)
# train_dataloader = DataLoader(train_data, batch_size=batch_size)

# validation_sampler = SequentialSampler(val_data)
validation_dataloader = DataLoader(val_data, batch_size=batch_size)

In [13]:
testset = Dataset_2(test_text, test_labels)
test_loader = DataLoader(testset, batch_size=8)

In [ ]:
acc_marich = []

for i in range(10):
    model = BertClassifier().to(device)
    tll, vll, tal, samp = marich(model,train_data,validation_dataloader,test_loader,budget = 60, init_points = 100, rounds = 6, epochs = 3, LR = 5e-6, model_name = "bert_attack"+str(i)+".pt", sampling = "all_elg", device = "cuda")

    acc_marich.append(tal)
    np.save("./results2/acc_marich_bert.npy", np.array(acc_marich))
    np.save("./results2/samples_bert.npy", np.array(samp))